In [1]:
#final runnable code 
from sklearn.svm import SVR 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
amazon = pd.read_csv('./company_stock_prices/AMZN.csv')

In [3]:
def SVR_past_Nvalues(data,N):
  y_lin_pred=[]
  y_poly_pred=[]
  y_rbf_pred=[]
  index = []
  
  print(data.shape)
  for i in range(N,len(data['Close'])):
    index.append(i)
    x_train=np.array(range(1,N+1)).reshape(-1,1)
    y_train=np.array(data['Close'][i-N:i])
    svr_lin  = SVR(kernel='linear', C=1e3)
    svr_poly = SVR(kernel='poly', C=1e3, degree=2)
    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    svr_lin.fit(x_train,y_train)
    svr_poly.fit(x_train,y_train)
    svr_rbf.fit(x_train,y_train)
    predicted_lin=svr_lin.predict(np.array(N+1).reshape(1,-1))[0]
    predicted_poly=svr_poly.predict(np.array(N+1).reshape(1,-1))[0]
    predicted_rbf=svr_rbf.predict(np.array(N+1).reshape(1,-1))[0]
    y_lin_pred.append(predicted_lin)
    y_poly_pred.append(predicted_poly)
    y_rbf_pred.append(predicted_rbf)
  # print(len(y_pred))
  return y_lin_pred,y_poly_pred,y_rbf_pred

In [4]:
def get_average_precision_score(y_test,y_pred):
  aps = average_precision_score(y_test, y_pred)
  return aps

In [5]:
def SVR_model1(data,N):
  pred_lin,pred_poly,pred_rbf=SVR_past_Nvalues(data,N)
  finalData=data[N:]
  finalData.reset_index(inplace = True, drop = True)
  finalData['PredictedClose(Linear)']=pd.DataFrame(pred_lin,columns=['PredictedClose(Linear)'])
  finalData['PredictedClose(Poly)']=pd.DataFrame(pred_poly,columns=['PredictedClose(Poly)'])
  finalData['PredictedClose(RBF)']=pd.DataFrame(pred_rbf,columns=['PredictedClose(RBF)'])
  return finalData

In [6]:
def getErrorScores(y_test,y_pred):
  mse = mean_squared_error(y_test,y_pred)
  # print('Testing Mean Squared Error: %.3f' % error)

  rmse = mean_squared_error(y_test, y_pred,squared=False)
  # print('Testing Root Mean Squared Error: %.3f' % error)

  mae = mean_absolute_error(y_test, y_pred)
  # print('Testing Mean absolute Error: %.3f' % error)

  r2 = r2_score(y_test, y_pred)
  # print('Testing Mean absolute Error: %.3f' % error)


  return mse,rmse,mae,r2

In [7]:
from sklearn.metrics import r2_score

In [8]:
def printErrorScores(test,pred):
  mse = mean_squared_error(test,pred)
  print('Testing Mean Squared Error: %.3f' % mse)

  rmse = mean_squared_error(test, pred,squared=False)
  print('Testing Root Mean Squared Error: %.3f' % rmse)

  mae = mean_absolute_error(test, pred)
  print('Testing Mean absolute Error: %.3f' % mae)

  r2 = r2_score(test, pred)
  print('Testing r2 score: %.3f' % r2)

In [9]:
def splits(data,train_ratio):
  train_length = int(len(data) * train_ratio)
  train_data = data[0:train_length]
  test_data = data[train_length+1:len(data)]
  return train_data,test_data

In [10]:
print("Amazon 30 days training window")
train_data,test_data=splits(amazon,0.7)
final_results_svr=SVR_model1(test_data,30)
final_results_svr
print('Errors for lienar kernel')
printErrorScores(final_results_svr['Close'],final_results_svr['PredictedClose(Linear)'])
print('Errors for Poly kernel')
printErrorScores(final_results_svr['Close'],final_results_svr['PredictedClose(Poly)'])
print('Errors for RBF kernel')
printErrorScores(final_results_svr['Close'],final_results_svr['PredictedClose(RBF)'])
final_results_svr
final_results_svr[['Close', 'PredictedClose(RBF)']].plot(color=['green', 'brown'])

Amazon 30 days training window
(377, 7)
